# Gaussian Quadrature

## Brief Introduction

Named after Carl Friedrich Gauss, Gaussian Quadrature is a method of evaluating a definite integral in an even more precise way when compared to other methods we've discussed in previous lectures, such as rectangle or trapezoidal. The method itself is quite similar to Simpson's Rule, albeit more precise considering the node placement for Simpson's rule are evenly spaced, whereas Gaussian Quadrature use a more dynamic placement for its nodes. 
Suppose we are given some function and wish to integrate it over the interval [-1,1], Gaussian Quadrature will give an estimated value of the integral based on the following, where $x_i$ and $\omega_i$ represent the nodes and weights respectively,

$$ \int_{-1}^{1}f(x)\thinspace dx = \sum_{i=1}^n\omega_if(x_i)$$ 


## Overview

As seen from the above summation, n represents the number of nodes, where 2n is the number of unknowns (nodes and weights) we need to solve for. With this in mind, we can state that the Gaussian Quadrature will be accurate for a polynomial of degree up to 2n-1. Expanding the summation, we get the following... 

$$
 \int_{-1}^{1}x^k\thinspace dx = \sum_{i=1}^n\omega_if(x_i) = \omega_1x_1^k + \omega_2x_2^k + ... + \omega_nx_n^k 
 $$ for $k=0,1,2,3,4,...,2n-1$.



Rigth away, we know this integral is symmetrical given the interval, and because of this, any value for k that happens to be odd, the integral simply reduces to 0. Now if k is even, we get...

$$ \int_{-1}^{1}x^k\thinspace dx = \frac{2}{k+1}\space, where\thinspace k \thinspace is\thinspace even $$

Now we could continue and attempt to solve for values of the nodes and weights, however, it is much easier and efficient to use legendre polynomials in the case where our interval is [-1,1], where the roots of the legendre polynomial will be coneniently used for our node values. Using the following, we have an expression for the weights, where $P'_n(x_i)$ is our legendre polynomial derivative

$$\omega_i = \frac{2}{\left(1-x_i^2\right)\left[P'_n(x_i)\right]^2}.$$


Using Bonnet's recursion formula, we can find values for all polynomials beyond $P_0$ and $P_1$, since their values are 1 and x respectively. The same goes for the recursion formula regarding the derivative of the legendre polynomial.

$$(n+1)P_{n+1} = (2n+1)xP_n - nP_{n-1}$$ 
$$(x^2-1)P'_n = nxP_n - nP_{n-1}.$$
where the polynomials $P_0 = 1$ and $P_1 = x$.

The expression we'll use to determine the roots of the polynomials is the following,
$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$



In [45]:
#The following code is used to determine the definite integral of a function by method of Gaussian Quadrature
#Used from lecture 5 notes for integreals being evaluated on the interval [-1,1], this particular example gives 
#the value for a simple function, cos(x).
import numpy as np
def g(x):
    return np.cos(x)
import scipy
def GaussQ(f,n):
    
    x = np.zeros(n,float)
    for i in range(n):
        x[i] = (1-(1/(8*n**2))+(1/(8*n**3)))*(np.cos((np.pi*(4*i-1))/(4*n+2)))
    
    epsilon = 1e-15
    delta = 1.0
    while delta>epsilon:
        p0 = np.ones(n,float)
        p1 = np.copy(x)
        for k in range(1,n):
       
            p0,p1 = p1,((2*k+1)*x*p1-k*p0)/(k+1)
            
        dp = (n+1)*(p0-x*p1)/(1-x*x)
        dx = p1/dp
        x -= dx
        delta = max(abs(dx))
    w = 2/((1-x**2)*dp*dp)
    return np.dot(f(x),w)

In [46]:
print(GaussQ(g,10000))

1.682605431703398


## Generalization

While the above method is accurate, its quite restrictive in its ability to be applied to any sort of varying bound conditions beyond -1 to 1. For this reason, it would be in our best interest to generalize the original formula to allow an integral to be evaluated on some interval [a,b], and it just so happens our original formula becomes the following...

$$\int_a^b f(x)\thinspace dx = \frac{b-a}{2}\sum_{i=1}^n \omega_i f\left(\frac{b-a}{2}x_i + \frac{a+b}{2}\right)$$ 

Using the expression to find the weights, as well as using the legendre polynomial roots as the nodes, we can evaluate this expression to give us a value for the definite integral, where the following code evaluates a simple cosine function on the interval [-2,5]. After, we'll compare Gaussian Quadrature and Simpson's Rule and the values these both produce on this interval for the function cosx. It should be noted that the amount of nodes for the gaussian quadrature method to converge on the integral value is far less when compared to Simpsons.

In [55]:
def f(x):
    return np.cos(x)

def GenGQ(f,n,a,b):
    x = np.zeros(n,float)
    for i in range(n):
        x[i] = (1-(1/(8*n**2))+(1/(8*n**3)))*(np.cos((np.pi*(4*i-1))/(4*n+2)))
        
    epsilon = 1e-15
    delta = 1.0
    while delta>epsilon:
        p0 = np.ones(n,float)
        p1 = np.copy(x)
        for k in range(1,n):
       
            p0,p1 = p1,((2*k+1)*x*p1-k*p0)/(k+1)
            
        dp = (n+1)*(p0-x*p1)/(1-x*x)
        dx = p1/dp
        x -= dx
        delta = max(abs(dx))
    w = 2/((1-x**2)*dp*dp)
    h = 1.0*b - 1.0*a
    z = 1.0*b + 1.0*a
    b = (0.5*h*x) + (z*0.5)
    num = np.dot(f(b),w)
    answer = h*num*0.5
    return answer


In [56]:
print(GenGQ(f,10000, -2,5))

-0.04961674225201264


## Gaussian Quadrature vs. Simpson's Rule

In [53]:
def Simpson(f,start, end, parts):
    deltax = (end - start) / parts
    resultsx = np.linspace(start, end, parts+1)
    resultsy = f(resultsx)
    area = np.empty([parts+1], float)
    for i in range(parts+1):
        if i == 0 or i == parts:
            coef = 1/3
        elif i%2 == 1:
            coef = 4/3
        else: 
            coef = 2/3
        area[i] = coef*resultsy[i]*deltax
    return sum(area)

In [54]:
print(Simpson(f,-2,5,10000))

-0.049626847837458216
